## Classify Articles:Three Steps
- Start with a corpus of articles
- Identify underlying themes
- Assign themes to new article


In [3]:
import urllib.request
from bs4 import BeautifulSoup

def getAllDoxyDonkeyPosts(url,links):
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    soup = BeautifulSoup(response)
    for a in soup.findAll('a'):
        try:
            url = a['href']
            title = a['title']
            if title == "Older Posts":
                print (title, url)
                links.append(url)
                getAllDoxyDonkeyPosts(url,links)
        except:
                 title = ""
    return
 
blogUrl = "http://doxydonkey.blogspot.in"
links = []
getAllDoxyDonkeyPosts(blogUrl,links)

Older Posts http://doxydonkey.blogspot.com/search?updated-max=2017-05-23T19:53:00-07:00&max-results=7
Older Posts http://doxydonkey.blogspot.com/search?updated-max=2017-05-14T19:02:00-07:00&max-results=7&start=7&by-date=false
Older Posts http://doxydonkey.blogspot.com/search?updated-max=2017-05-02T19:43:00-07:00&max-results=7&start=14&by-date=false
Older Posts http://doxydonkey.blogspot.com/search?updated-max=2017-04-17T19:26:00-07:00&max-results=7&start=21&by-date=false
Older Posts http://doxydonkey.blogspot.com/search?updated-max=2017-04-10T18:56:00-07:00&max-results=7&start=28&by-date=false
Older Posts http://doxydonkey.blogspot.com/search?updated-max=2017-03-30T19:57:00-07:00&max-results=7&start=35&by-date=false
Older Posts http://doxydonkey.blogspot.com/search?updated-max=2017-03-20T19:47:00-07:00&max-results=7&start=42&by-date=false
Older Posts http://doxydonkey.blogspot.com/search?updated-max=2017-03-02T17:42:00-08:00&max-results=7&start=49&by-date=false
Older Posts http://doxyd

Older Posts http://doxydonkey.blogspot.com/search?updated-max=2015-04-23T20:19:00-07:00&max-results=7&start=462&by-date=false
Older Posts http://doxydonkey.blogspot.com/search?updated-max=2015-04-14T19:40:00-07:00&max-results=7&start=469&by-date=false
Older Posts http://doxydonkey.blogspot.com/search?updated-max=2015-04-05T20:22:00-07:00&max-results=7&start=476&by-date=false
Older Posts http://doxydonkey.blogspot.com/search?updated-max=2015-03-24T20:12:00-07:00&max-results=7&start=483&by-date=false
Older Posts http://doxydonkey.blogspot.com/search?updated-max=2015-03-15T20:41:00-07:00&max-results=7&start=490&by-date=false
Older Posts http://doxydonkey.blogspot.com/search?updated-max=2015-03-03T19:30:00-08:00&max-results=7&start=497&by-date=false
Older Posts http://doxydonkey.blogspot.com/search?updated-max=2015-02-22T19:55:00-08:00&max-results=7&start=504&by-date=false
Older Posts http://doxydonkey.blogspot.com/search?updated-max=2015-02-11T20:02:00-08:00&max-results=7&start=511&by-dat

In [15]:
def getDoxyDonkeyText(testUrl):
    request = urllib.request.Request(testUrl)
    response = urllib.request.urlopen(request)
    soup = BeautifulSoup(response)
    #each post has class post-body, within this div we need to find bullet point having tag "<li>"
    mydivs = soup.findAll("div", {"class":'post-body'})
    #empty list to collect all articles
    posts =[]
    for div in mydivs:
        posts+=map(lambda p:p.text.encode('ascii', errors='replace').decode().replace("?"," "), div.findAll("li"))
    #return all the articles in this url
    #return str(text.encode('ascii', errors='replace')).replace("?", " ")
    return posts

doxyDonkeyPosts = []
for link in links:
    doxyDonkeyPosts+=getDoxyDonkeyText(link)

#print all the articles contained in the list doxyDonkeyPosts
doxyDonkeyPosts

["SoftBank's $100 Billion Tech Fund Rankles VCs as Valuations Soar: In the months since Softbank Group Corp. unveiled plans for a $100 billion technology fund, the Japanese company has been making its presence deeply felt across the industry. The Vision Fund closed a few days ago with $93 billion in initial commitments, and already venture firms from London to Silicon Valley are fretting about a behemoth with the resources, clout and name recognition to snatch away the most promising deals. Just last week, SoftBank swooped in and pumped $1.4 billion into Paytm, India s largest digital-payments startup. The deal boosted Paytm's valuation by about 40 percent to $7 billion. That's not outlandish given Paytm's dominant market position, but the valuations of other SoftBank deals have prompted head-scratching and ignited alarm that a funding atmosphere that only recently cooled off will heat up again. there's the concern that SoftBank will ladle out more money than startups need or can absor

In [16]:
#Convert text to TF-IDF representation
from sklearn.feature_extraction.text import TfidfVectorizer

#By setting stopwords to relevant language, it will ignore the stopwords
vectorizer = TfidfVectorizer(max_df=0.5,min_df=2,stop_words='english')

X = vectorizer.fit_transform(doxyDonkeyPosts)
X


<2804x13220 sparse matrix of type '<class 'numpy.float64'>'
	with 280835 stored elements in Compressed Sparse Row format>

In [18]:
print (X[0])

  (0, 10926)	0.5733279543023014
  (0, 14)	0.09280505849401514
  (0, 1742)	0.13328128802771985
  (0, 11780)	0.039275583841741034
  (0, 5218)	0.15659973720628836
  (0, 12637)	0.08633297199995174
  (0, 12609)	0.12106620886264521
  (0, 10912)	0.08963771830239645
  (0, 7712)	0.03953053864890453
  (0, 5524)	0.04161080105144049
  (0, 3101)	0.04935454181581896
  (0, 12512)	0.06594121878805018
  (0, 8781)	0.042540202954173906
  (0, 11794)	0.0344321913352514
  (0, 6546)	0.06616404362598743
  (0, 7261)	0.04323887970998493
  (0, 9012)	0.06616404362598743
  (0, 3476)	0.08734850508251557
  (0, 4820)	0.0801392274857223
  (0, 6196)	0.04077563193498788
  (0, 12739)	0.07064073666904815
  (0, 2618)	0.05966733705364259
  (0, 3398)	0.05187934981734117
  (0, 882)	0.0422233466274211
  (0, 575)	0.07683448118327758
  :	:
  (0, 11730)	0.06888898157373023
  (0, 12211)	0.06594121878805018
  (0, 7910)	0.08734850508251557
  (0, 8623)	0.04928881782605509
  (0, 10302)	0.04266093876248732
  (0, 13048)	0.05347359253941

In [19]:
X[0]

from sklearn.cluster import KMeans

#max_iter means if algorithm don't converge even after this no# of iteration, it will stop
km = KMeans(n_clusters = 3, init = 'k-means++', max_iter = 100, n_init = 1, verbose = True)
km.fit(X)

Initialization complete
Iteration  0, inertia 5260.700
Iteration  1, inertia 2685.814
Iteration  2, inertia 2678.721
Iteration  3, inertia 2676.366
Iteration  4, inertia 2675.097
Iteration  5, inertia 2674.360
Iteration  6, inertia 2673.892
Iteration  7, inertia 2673.403
Iteration  8, inertia 2672.809
Iteration  9, inertia 2672.381
Iteration 10, inertia 2672.105
Iteration 11, inertia 2671.945
Iteration 12, inertia 2671.827
Iteration 13, inertia 2671.707
Iteration 14, inertia 2671.575
Iteration 15, inertia 2671.498
Iteration 16, inertia 2671.449
Iteration 17, inertia 2671.422
Iteration 18, inertia 2671.400
Iteration 19, inertia 2671.391
Iteration 20, inertia 2671.387
Iteration 21, inertia 2671.382
Iteration 22, inertia 2671.376
Iteration 23, inertia 2671.369
Iteration 24, inertia 2671.367
Iteration 25, inertia 2671.364
Iteration 26, inertia 2671.360
Iteration 27, inertia 2671.353
Iteration 28, inertia 2671.351
Iteration 29, inertia 2671.349
Iteration 30, inertia 2671.347
Converged at it

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=3, n_init=1, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=True)

In [20]:
import numpy as np

#km.lables_ is an attribute of K-means object. unique function will print the unique clusters name with each article have been assign one cluster
np.unique(km.labels_, return_counts=True)

(array([0, 1, 2]), array([1615,  312,  877], dtype=int64))

In [21]:
#Aggregate text in each cluster
text={}
for i,cluster in enumerate(km.labels_):
        oneDocument = doxyDonkeyPosts[i]
        if cluster not in text.keys():
            text[cluster] = oneDocument
        else:
            text[cluster] += oneDocument

In [22]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict
from string import punctuation
from heapq import nlargest
import nltk

_stopwords = set(stopwords.words('english') + list(punctuation)+["million","billion","year","millions","billions","y/y","'s","''","``"])

#Top keywords in each cluster and their count
keywords = {}
counts={}
for cluster in range(3):
    word_sent = word_tokenize(text[cluster].lower())
    word_sent=[word for word in word_sent if word not in _stopwords]
    freq = FreqDist(word_sent)
    keywords[cluster] = nlargest(100, freq, key=freq.get)
    counts[cluster]=freq


                                                                 
                                        

In [23]:
unique_keys={}
for cluster in range(3):
    other_clusters=list(set(range(3))-set([cluster]))
    #collect all the keywords from other clusters
    keys_other_clusters=set(keywords[other_clusters[0]]).union(set(keywords[other_clusters[1]]))
    #Take out these keywords from the whole set to get unique keywords
    unique=set(keywords[cluster])-keys_other_clusters
    #Store top 10 keywords 
    unique_keys[cluster]=nlargest(10, unique, key=counts[cluster].get)

In [24]:
#After analysing words in each cluster we can identify the theme of each cluster
unique_keys

{0: ['twitter',
  'video',
  'get',
  'way',
  'social',
  'ad',
  'platform',
  'using',
  'site',
  'big'],
 1: ['apple',
  'iphone',
  'watch',
  'samsung',
  'android',
  'devices',
  'smartphone',
  'store',
  'phone',
  'system'],
 2: ['investors',
  'alibaba',
  'stock',
  'profit',
  'public',
  'valuation',
  'round',
  'rose',
  'per',
  'earnings']}

In [26]:
article = "Alibaba Group Holding Ltd.’s six-year-old excursion into mobile operating systems is faltering in China, casting doubt over software that bears billionaire-founder Jack Ma’s name and was once touted as key to countering Tencent Holdings Ltd. China’s largest e-commerce company debuted YunOS in 2011, a system that underpins search, shopping and browsing that its executives last year said could attain as much as 25 percent domestic market share by the end of 2016 -- surpassing Apple Inc.’s iOS. Six years on, YunOS’ slice of China software installations stands at just 2.2 percent while its share of 2016 shipments was 10 percent, researchers Canalys and Counterpoint estimate, respectively. Alibaba disputes those numbers. Alibaba managers have grown increasingly unhappy with its sluggish adoption and have begun an internal debate around the software’s future, a person familiar with the matter said. No conclusions have been reached, the person said, asking not to be named discussing a confidential matter. Yet the talks reflect the inability of a once-vaunted initiative to forestall Tencent’s dominance in the mobile arena, secured through the utility of WeChat -- a universal app that melds messaging, payments, media, shopping and on-demand services."
from sklearn.neighbors import KNeighborsClassifier

#here K(n_neighbors)=5 by default
classifier = KNeighborsClassifier(n_neighbors=3)

#training phase
classifier.fit(X,km.labels_)
test=vectorizer.transform([article.encode('ascii',errors='ignore')])
classifier.predict(test)

array([0])